In [72]:
import sys
sys.path.append("../")

from typing import Callable, Tuple
from fppy.gt import S, T, GenericTypeVar

VoidCallable = Callable[[], S]

def lazy_cons(head: VoidCallable[S], tail: VoidCallable[S] | Tuple):
    """基于二元元组的LazyList
    """
    return (head, tail)

def make_lazy_cons(*args: S) -> VoidCallable[S]:
    if len(args) == 0:
        return ()
    else:
        return lazy_cons(lambda: args[0], lambda: make_lazy_cons(*args[1:]))

head = lambda lls: lls[0]()
tail = lambda lls: lls[1]()

def iterate_lazy_cons(start: S):
    def helper(f: Callable[[S], S]):
        return lazy_cons(
            lambda: start, 
            lambda: iterate_lazy_cons(f(start))(f)
        )
    return helper

def filter_lazy_cons(f: Callable[[S], bool]):
    def helper(lls):
        if lls == ():
            return ()
        else:
            return lazy_cons(
                lambda : head(lls) if head(lls) != () and f(head(lls)) else (),
                lambda : filter_lazy_cons(f)(tail(lls))
            )
    return helper

def collect_lazy_cons(lls):
    if lls == ():
        return ()
    elif head(lls) == ():
        return collect_lazy_cons(tail(lls))
    else:
        return (head(lls), collect_lazy_cons(tail(lls)))

def take_lazy_cons(n: int):
    def helper(lls):
        if n == 0:
            return ()
        elif head(lls) == ():
            return take_lazy_cons(n)(tail(lls))
        else:
            return (head(lls), take_lazy_cons(n - 1)(tail(lls)))
    return helper

In [73]:
a = iterate_lazy_cons(1)(lambda x: x + 1)

In [74]:
take_lazy_cons(3)(a)

(1, (2, (3, ())))

In [75]:
b = make_lazy_cons(1, 2, 3)

In [76]:
collect_lazy_cons(filter_lazy_cons(lambda x: x % 2 == 1)(b))

(1, (3, ()))

In [77]:
take_lazy_cons(2)(b)

(1, (2, ()))

In [78]:
collect_lazy_cons(b)

(1, (2, (3, ())))

In [79]:
def map_lazy_cons(f):
    def helper(lls):
        if lls == ():
            return ()
        if head(lls) == ():
            return lazy_cons(
                lambda: (),
                lambda: map_lazy_cons(f)(tail(lls))
            )
        else:
            return lazy_cons(
                lambda: f(head(lls)),
                lambda: map_lazy_cons(f)(tail(lls))
            )
    return helper

In [83]:
take_lazy_cons(4)(map_lazy_cons(lambda x: x+ 1)(a))

(2, (3, (4, (5, ()))))